<a href="https://colab.research.google.com/github/soul11zz/imagecaption/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Train with Pytorch Lighning

In [1]:
import os
import os.path as osp
if osp.exists("imagecaption"):
    os.chdir("imagecaption")

In [2]:
!pip install -r train/requirements.txt

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-4csdobh4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-4csdobh4
  Resolved https://github.com/huggingface/transformers.git to commit 73a2ff69740123ef85343580cbfa9ee8ce5e6fd5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 20.9 MB/s eta 0:00:00


In [4]:
import os.path as osp
import sys
train_dir = osp.abspath("train")
if train_dir not in sys.path:
    sys.path.append(train_dir)
    
import pl_module
import pytorch_lightning as pl
from huggingface_hub import HfApi, notebook_login

pl.seed_everything(42, workers=True)


Global seed set to 42


42

### Login to HuggingFace

In [5]:

notebook_login()

### Load the model

In [6]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from pl_module import ImageCaptioningModule
from dataset import ImageCaptioningDataset
from transformers import GitProcessor, GitForCausalLM

from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

out_model = dataset_desc_only = "soul11zz/image-caption-desc-only"
in_model = "microsoft/git-large"
# model_name = get_input_model_name(out_model)

dt_train = load_dataset(dataset_desc_only, split="train")
dt_val = load_dataset(dataset_desc_only, split="validation")
dt_test = load_dataset(dataset_desc_only, split="test")

processor = GitProcessor.from_pretrained(in_model)
train_dataset = ImageCaptioningDataset(dt_train, processor)
val_dataset = ImageCaptioningDataset(dt_val, processor)
test_dataset = ImageCaptioningDataset(dt_test, processor)


2023-01-29-05-42-50  WARNING    Using custom data configuration soul11zz--image-caption-desc-only-7581b595eec46650


2023-01-29-05-44-04  WARNING    Computing checksums of downloaded files. They can be used for integrity verification. You can disable this by passing ignore_verifications=True to load_dataset


Computing checksums:  45%|####5     | 5/11 [00:05<00:06,  1.16s/it]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/13065 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2306 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2713 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/soul11zz___parquet/soul11zz--image-caption-desc-only-7581b595eec46650/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


2023-01-29-05-44-30  WARNING    Using custom data configuration soul11zz--image-caption-desc-only-7581b595eec46650
2023-01-29-05-44-30  WARNING    Found cached dataset parquet (/root/.cache/huggingface/datasets/soul11zz___parquet/soul11zz--image-caption-desc-only-7581b595eec46650/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
2023-01-29-05-44-30  WARNING    Using custom data configuration soul11zz--image-caption-desc-only-7581b595eec46650
2023-01-29-05-44-30  WARNING    Found cached dataset parquet (/root/.cache/huggingface/datasets/soul11zz___parquet/soul11zz--image-caption-desc-only-7581b595eec46650/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


### Prelims for PL Module

In [26]:
batch_size = 1

model_dir = "tmp/model"
epochs = 10

if not osp.exists(model_dir):
  os.makedirs(model_dir)
  
num_workers = os.cpu_count() if os.name != "nt" else 0
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=num_workers)

model = GitForCausalLM.from_pretrained(in_model)
callbacks = []

pl_train_module = ImageCaptioningModule(processor, model, train_loader, val_loader, test_loader, learning_rate=1e-2)

### Trainer
logger = TensorBoardLogger("tb_logs", name="image-captioning")

checkpoint = ModelCheckpoint(dirpath=model_dir, 
                              save_top_k=2, monitor="val_loss", 
                              mode="min", 
                              filename="imcap-{epoch:02d}-{val_loss:.2f}")

callbacks += [checkpoint]


### Create PL Trainer

In [27]:
trainer = pl.Trainer( 
                      logger=logger, 
                      gpus=1,
                      callbacks=callbacks,
                      max_epochs=epochs,
                      check_val_every_n_epoch=1,
                      # val_check_interval=50,
                      precision=16,
                      num_sanity_val_steps=1,
                      )


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### (optional) Tune Learning Rate

In [28]:
tuner = pl.Trainer(auto_lr_find=True, accelerator="cuda", devices=1, max_epochs=1)
tuner.tune(pl_train_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RuntimeError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 23.69 GiB total capacity; 21.57 GiB already allocated; 14.56 MiB free; 21.88 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
pl_train_module.lr


0.0001

### Run Training

In [ ]:
trainer.fit(pl_train_module)

### Upload Best Model

In [24]:
pl_model_best = ImageCaptioningModule.load_from_checkpoint(checkpoint.best_model_path, processor=processor, model=model, train_dataloader=train_loader, val_dataloader=val_loader)
pl_model_best.model.save_pretrained("tb_logs/image-captioning/best_model", push_to_hub=True, repo_id=out_model)
pl_model_best.processor.save_pretrained("tb_logs/image-captioning/best_model", push_to_hub=True, repo_id=out_model)


pytorch_model.bin:   0%|          | 0.00/707M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

### Evaluate Best Model

In [ ]:
trainer.test(pl_train_module)